### MA6202: Laboratorio de Ciencia de Datos

#### Profesor: Nicolás Caro

#### Fecha de entrega: 17/05/2020

#### Integrantes: Matías Romero, Danner Schlotterbeck, Kurt Walsen

# Tarea 1

In [1]:
import pandas as pd
import numpy as np

# P1 Limpieza de datos

# 1

In [2]:
# Se carga una de las semanas de la data para inspeccionar:
w13_all = pd.read_csv('./data/raw/w{}/metrocuadrado_all_w{}.csv'.format(13,13))
w13_fur= pd.read_csv('./data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(13,13))

In [3]:
# Inspeccionamos sus columnas, y notamos que son las mismas
print(w13_all.columns)
print(w13_fur.columns)
w13_all.columns==w13_fur.columns

Index(['property_type|rent_type|location', 'price', 'n_rooms', 'n_bath',
       'surface', 'details', 'url', 'metrocuadrado_index'],
      dtype='object')
Index(['property_type|rent_type|location', 'price', 'n_rooms', 'n_bath',
       'surface', 'details', 'url', 'metrocuadrado_index'],
      dtype='object')


array([ True,  True,  True,  True,  True,  True,  True,  True])

In [4]:
'''Realizamos una concatenación con y sin duplicados para verificar la 
existencia de éstos.
'''
df_con=pd.concat([w13_all,w13_fur],ignore_index=True)
df_con

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1
...,...,...,...,...,...,...,...,...
13612,"Apartamento en Arriendo, SANTA BARBARA OCCIDEN...",$4.500.000,2,3,136.0 m2,"EXCELENTE APARTAMENTO AMOBLADO, ALCOBAS CON PI...",https://www.metrocuadrado.com/inmueble/arriend...,64.1
13613,"Apartamento en Arriendo, SANTA BARBARA OCCIDEN...",$4.390.000,2,3,110.0 m2,amoblado con terraza.,https://www.metrocuadrado.com/inmueble/arriend...,64.2
13614,"Apartamento en Arriendo, ANTIGUO COUNTRY Bogot...",$4.500.000,2,3,120.0 m2,"Lindo apto en el sector de Chico Virrey, cerca...",https://www.metrocuadrado.com/inmueble/arriend...,55.4
13615,"Apartamento en Arriendo, MOLINOS NORTE Bogotá ...",$4.100.000,2,3,114.0 m2,"Hermoso apto, rodeado de verde por todos lados...",https://www.metrocuadrado.com/inmueble/arriend...,59.1


In [5]:
# Verificamos si existen duplicados
df_con.shape[0]==df_con.drop_duplicates().shape[0] # False, luego existen.

False

In [6]:
'''
Buscamos generar una variable categórica que indique si la observación
correspondiente proviene de un archivo que en su nombre contiene `furnished`.
Para ello haremos uso de la función merge.

Notamos que la columna que sirve de identificador es `url` pues debiese 
ser único para cada propiedad en arriendo. Además podemos utilizarlo como llave
para hacer el merge de los dataframes y ver qué ocurre con al columna
`furnished`.

De esta manera identificamos a archivos de texto `furnished` que no estén
contenidos en archivos con texto `all` mediante la inspección de los valores
right_only en la columna 'furnished'.
'''
w13_all = w13_all.drop_duplicates()
w13_fur = w13_fur.drop_duplicates()


df_mer = pd.merge(left=w13_all,right=w13_fur,on='url',how='outer',indicator='furnished')
df_mer

,property_type|rent_type|location_x,price_x,n_rooms_x,n_bath_x,surface_x,details_x,url,metrocuadrado_index_x,property_type|rent_type|location_y,price_y,n_rooms_y,n_bath_y,surface_y,details_y,metrocuadrado_index_y,furnished
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3.0,2.0,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,57.3,both
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11956,NaN,NaN,NaN,NaN,NaN,NaN,https://www.metrocuadrado.com/inmueble/arriend...,NaN,"Casa en Arriendo, SANTA HELENA I Bogotá D.C..",$3.233.000,3.0,4.0,202.0 m2,Casa tres niveles conjunto cerrado. Área de 20...,25.2,right_only
11957,NaN,NaN,NaN,NaN,NaN,NaN,https://www.metrocuadrado.com/inmueble/arriend...,NaN,"Casa en Arriendo, ATAHUALPA Bogotá D.C..",$5.500.000,4.0,3.0,220.0 m2,"Hermosa casa para sede educativa,o fundación,d...",66.8,right_only
11958,NaN,NaN,NaN,NaN,NaN,NaN,https://www.metrocuadrado.com/inmueble/arriend...,NaN,"Casa en Arriendo, JULIO FLOREZ Bogotá D.C..",$2.900.000,4.0,4.0,216.0 m2,Arriendo Casa La Floresta en dos Niveles y un ...,33.8,right_only
11959,NaN,NaN,NaN,NaN,NaN,NaN,https://www.metrocuadrado.com/inmueble/arriend...,NaN,"Apartamento en Arriendo, SANTA BIBIANA Bogotá ...",$1.900.000,1.0,2.0,47.0 m2,Aparta-estudio amoblado en arriendo sector res...,37.5,right_only


In [7]:
'''Para w13 podemos notar 5 de éstos casos. Pero hay que verlo para el caso
general.
'''
fur=df_mer.drop_duplicates()[['url','furnished']]
fur[fur['furnished']=='right_only']

,url,furnished
11956,https://www.metrocuadrado.com/inmueble/arriend...,right_only
11957,https://www.metrocuadrado.com/inmueble/arriend...,right_only
11958,https://www.metrocuadrado.com/inmueble/arriend...,right_only
11959,https://www.metrocuadrado.com/inmueble/arriend...,right_only
11960,https://www.metrocuadrado.com/inmueble/arriend...,right_only


In [8]:
'''Ahora, agregamos al dataframe de concatenación la columna con la variable
categórica furnished, mediante el uso de
'''
df_con = df_con.drop_duplicates()
pd.merge(df_con,fur,on='url').drop_duplicates()

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,furnished
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,left_only
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,both
2,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,both
3,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,left_only
4,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,left_only
...,...,...,...,...,...,...,...,...,...
13561,"Casa en Arriendo, SANTA HELENA I Bogotá D.C..",$3.233.000,3,4,202.0 m2,Casa tres niveles conjunto cerrado. Área de 20...,https://www.metrocuadrado.com/inmueble/arriend...,25.2,right_only
13562,"Casa en Arriendo, ATAHUALPA Bogotá D.C..",$5.500.000,4,3,220.0 m2,"Hermosa casa para sede educativa,o fundación,d...",https://www.metrocuadrado.com/inmueble/arriend...,66.8,right_only
13563,"Casa en Arriendo, JULIO FLOREZ Bogotá D.C..",$2.900.000,4,4,216.0 m2,Arriendo Casa La Floresta en dos Niveles y un ...,https://www.metrocuadrado.com/inmueble/arriend...,33.8,right_only
13564,"Apartamento en Arriendo, SANTA BIBIANA Bogotá ...",$1.900.000,1,2,47.0 m2,Aparta-estudio amoblado en arriendo sector res...,https://www.metrocuadrado.com/inmueble/arriend...,37.5,right_only


In [9]:
'''Se compara con el DataFrame sin la columna agregada, se observa que poseen
la misma cantidad de registros.
'''
df_con

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1
...,...,...,...,...,...,...,...,...
13612,"Apartamento en Arriendo, SANTA BARBARA OCCIDEN...",$4.500.000,2,3,136.0 m2,"EXCELENTE APARTAMENTO AMOBLADO, ALCOBAS CON PI...",https://www.metrocuadrado.com/inmueble/arriend...,64.1
13613,"Apartamento en Arriendo, SANTA BARBARA OCCIDEN...",$4.390.000,2,3,110.0 m2,amoblado con terraza.,https://www.metrocuadrado.com/inmueble/arriend...,64.2
13614,"Apartamento en Arriendo, ANTIGUO COUNTRY Bogot...",$4.500.000,2,3,120.0 m2,"Lindo apto en el sector de Chico Virrey, cerca...",https://www.metrocuadrado.com/inmueble/arriend...,55.4
13615,"Apartamento en Arriendo, MOLINOS NORTE Bogotá ...",$4.100.000,2,3,114.0 m2,"Hermoso apto, rodeado de verde por todos lados...",https://www.metrocuadrado.com/inmueble/arriend...,59.1


In [10]:
'''Ahora, realizamos el proceso general para todas las semanas disponibles
en la data.
'''
all_dfs = []
fur_dfs = []
for i in range(13,18):
    df_all = pd.read_csv('./data/raw/w{}/metrocuadrado_all_w{}.csv'.format(i,i))
    df_fur = pd.read_csv('./data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(i,i))
    all_dfs.append(df_all)
    fur_dfs.append(df_fur)

df_all_concat = all_dfs[0] 
df_fur_concat = fur_dfs[0]
n=len(all_dfs)
print('Número de semanas incluidas: ', n)

for j in range(1,n):
    df_all_concat = pd.concat([df_all_concat,all_dfs[j]])
    df_fur_concat = pd.concat([df_fur_concat,fur_dfs[j]])
    
df_all_concat = df_all_concat.drop_duplicates()
df_fur_concat = df_fur_concat.drop_duplicates()

df_concat = pd.concat([df_all_concat,df_fur_concat]).drop_duplicates()

df_merged = pd.merge(df_all_concat,df_fur_concat,on='url',how='outer',indicator='furnished')
furnished = df_merged[['url','furnished']]  

only_fur= furnished[furnished['furnished']=='right_only']

print("Observaciones de archivos con texto 'furnished' que no estén contenidos en archivos con texto 'all': ",
      len(only_fur))

df = pd.merge(df_concat,furnished,on='url').drop_duplicates()

Número de semanas incluidas:  5
Observaciones de archivos con texto 'furnished' que no estén contenidos en archivos con texto 'all':  4


In [11]:
df

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,furnished
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,left_only
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,both
2,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,both
3,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,left_only
4,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,left_only
...,...,...,...,...,...,...,...,...,...
23206,"Apartamento en Arriendo, SAN PATRICIO Bogotá D...",$1.550.000,1,2,56.0 m2,"ESPECTACULAR DUPLEX, APARTA-ESTUDIO, EXTERIOR,...",https://www.metrocuadrado.com/inmueble/arriend...,42.2,left_only
23207,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.750.000,3,2,117.0 m2,Hermosa casa amoblada en arriendo ubicada en e...,https://www.metrocuadrado.com/inmueble/arriend...,66.4,right_only
23208,"Apartamento en Arriendo, LOS CEDRITOS Bogotá D...",$2.300.000,1,1,48.0 m2,Arriendo apartamento amoblado nuevo moderno ti...,https://www.metrocuadrado.com/inmueble/arriend...,72.7,right_only
23209,"Apartamento en Arriendo, SANTA BIBIANA Bogotá ...",$2.600.000,1,2,60.0 m2,"Se arrienda precioso apartamento amoblado, con...",https://www.metrocuadrado.com/inmueble/arriend...,63.4,right_only


In [12]:
'''Se compara con el DataFrame sin la columna agregada, se observa que poseen
la misma cantidad de registros.
'''
df_concat

,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1
...,...,...,...,...,...,...,...,...
1225,"Apartamento en Arriendo, CHICO NORTE Bogotá D.C..",$4.620.000,2,3,150.0 m2,SE ARRIENDA HERMOSO APARTAMENTO AMOBLADO DE 15...,https://www.metrocuadrado.com/inmueble/arriend...,55.6
1232,"Apartamento en Arriendo, CHICO NORTE II Bogotá...",$2.700.000,2,2,164.34 m2,Espectacular apartamento en exclusiva zona res...,https://www.metrocuadrado.com/inmueble/arriend...,26.1
1302,"Apartamento en Venta Y Arriendo, CHICO NORTE B...",$3.600.000,2,3,110.0 m2,Espectacular apartamento DUPLEX muy iluminado ...,https://www.metrocuadrado.com/inmueble/venta-a...,50.8
1383,"Apartamento en Arriendo, MOLINOS NORTE Bogotá ...",$2.525.000,2,3,106.0 m2,Hermoso apartamento de 106m2 2 añoS de const...,https://www.metrocuadrado.com/inmueble/arriend...,44.4


# 2